In [2]:
from cde.density_estimator import KernelMixtureNetwork, MixtureDensityNetwork#, NormalizingFlowEstimator
import pandas as pd
import numpy as np 
import argparse 
from tqdm import tqdm 

/home/ruijiang/anaconda3/envs/balancecde/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ruijiang/anaconda3/envs/balancecde/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ruijiang/anaconda3/envs/balancecde/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ruijiang/anaconda3/env

In [16]:
sen_feature = 'sex_Male_1.0'
sens_feature = sen_feature
immutable = ['sex_1']
mutable = ['x1','x2']
all_data = pd.read_csv('./balance_data/syn_train.csv', index_col = 0)
all_data.columns = ['x1','x2','sex_1', 'y']
negative_data = pd.read_csv('./balance_data/syn_negative_instances.csv', index_col = 0)
positive_data = all_data[~all_data.index.isin(negative_data.index)]

def rej_sampling_syn(negative_data):
    # sampling high density samples for both male and female 
    sampled_points = np.empty(shape = (0, len(mutable)))
    density = []
    # for each point sample 30 responses 
    while sampled_points.shape[0] < 1000:
        print(sampled_points.shape)
        sample_size = 200000
        _, thissample = density_model.sample(np.ones(sample_size))
        uniform = np.random.uniform(size = sample_size)
        accepted_samples = thissample[uniform < density_model.pdf(np.zeros(sample_size), thissample)]
        sampled_points = np.concatenate((sampled_points, accepted_samples))
    samples_density = density_model.pdf(np.zeros(sampled_points.shape[0]),sampled_points) * \
        density_model.pdf(np.ones(sampled_points.shape[0]),sampled_points)
    sampled_data = pd.DataFrame(sampled_points)
    sampled_data.columns = mutable 
    sampled_data['density'] = samples_density 
    sampled_data.to_csv(file_name)
    return sampled_data

density_model = MixtureDensityNetwork(f'mixd111', len(immutable), len(mutable), n_centers = 50, hidden_sizes = (16,16), n_training_epochs = 1000)
all_data = positive_data.append(negative_data)
file_name = f'./balance_data/sampled_data/balance_sampling.csv'
density_model.fit(positive_data[immutable].values, positive_data[mutable].values)

rej_sampling_syn(negative_data)

1000/1000 [100%] ██████████████████████████████ Elapsed: 18s | Loss: 540.453
mean log-loss train: 1.070
(0, 2)


,x1,x2,density
0,0.581395,0.644522,3.794914
1,0.526088,0.662963,8.072914
2,0.656860,0.751283,22.288120
3,0.766761,0.658677,0.077408
4,0.700903,0.747241,8.541800
...,...,...,...
34586,0.565414,0.691424,2.039733
34587,0.630436,0.612625,3.383236
34588,0.452240,0.567722,0.020383
34589,0.577186,0.614372,5.765669
